In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt

In [2]:
from bs4 import BeautifulSoup, SoupStrainer
import requests

#### ONC 2019

In [5]:
url = "https://onciencias.org/resultado/resultado-final-onc-2019/"
url_home = 'https://onciencias.org/'

page = requests.get(url)    
data = page.text
soup = BeautifulSoup(data)

links = []

for link in soup.find_all('a'):
    temp_link = link.get('href')
    if(str(temp_link).find('medalha') != -1 or str(temp_link).find('mencao-honrosa') != -1):
        links.append(url_home+temp_link[1:])
        #print(url_home+temp_link[1:])
        
serie = []
medalhas = []
for link in links:
    temp_splited_link = link.split('/')
    serie.append(temp_splited_link[-2])
    medalhas.append(temp_splited_link[-1])


In [6]:
dfs = []
for a,b,c in zip(serie,medalhas,links):
    df = pd.read_html(c)[0]
    df['serie'] = a
    df['medalha'] = b
    dfs.append(df)
appended_data = pd.concat(dfs)

In [15]:
appended_data.head()

,Nome do Aluno,Escola,Estado,Cidade,Unnamed: 4,serie,medalha
0,LARA DANTAS DE OLIVEIRA MOISES,7 DE SETEMBRO COLEGIO - NGS,CEARÁ,Fortaleza,NaN,9-ano-ensino-fundamental,medalha-ouro
1,NAILTON GAMA DE CASTRO A,MASTER COLEGIO,CEARÁ,Fortaleza,NaN,9-ano-ensino-fundamental,medalha-ouro
2,RAFAEL MORENO RIBEIRO,COLEGIO MILITAR DE SALVADOR,BAHIA,Salvador,NaN,9-ano-ensino-fundamental,medalha-ouro
3,ARTHUR PINTO LORENZO,COLEGIO MILITAR DE SALVADOR,BAHIA,Salvador,NaN,9-ano-ensino-fundamental,medalha-ouro
4,GUILHERME Z. HAUSSEN,COLEGIO ANCHIETA,RIO GRANDE DO SUL,Porto Alegre,NaN,9-ano-ensino-fundamental,medalha-ouro


#### ONC 2018

In [3]:
links_onc_2018= ['http://onciencias.org/resultados/download/15',
                'http://onciencias.org/resultados/download/16',
                'http://onciencias.org/resultados/download/17',
                'http://onciencias.org/resultados/download/18']

#### ONC 2016

In [26]:
links_onc_2016 = ['https://onciencias.org/resultados/download/9/ResultadoONC2016-Nivel%20D.pdf',
'https://onciencias.org/resultados/download/8/ResultadoONC2016-Nivel%20C.pdf',
'https://onciencias.org/resultados/download/7/ResultadoONC2016-Nivel%20B.pdf',
'https://onciencias.org/resultados/download/6/ResultadoONC2016-Nivel%20A.pdf']

In [51]:
rr = download_file(links_onc_2018[0])
print(rr)

None


In [40]:
def download_file(url):
    local_filename = url.split('/')[-1]
    r = requests.get(url)
    f = open(local_filename, 'wb')
    for chunk in r.iter_content(chunk_size=512 * 1024): 
        if chunk: # filter out keep-alive new chunks
            f.write(chunk)
    f.close()
    return local_filename